In [1]:
#
#     The data used in this notebook was cleaned by the script by MadScientist:
#     https://www.kaggle.com/keremt/very-extensive-cleaning-by-sberbank-discussions
#
#

import pandas as pd
import numpy as np
import lightgbm as lgb

def preprocess(data):

    #add relative floor
    data['rel_floor'] = data['floor']/data['max_floor']
    #add relative kitch_sq
    data['relative_kitch_sq'] = data['kitch_sq']/data['full_sq']
    #add room size
    data['room_size'] = data['life_sq']/data['num_room']
    # add month and day of week
    data['month'] = data.timestamp.dt.month.astype(int)
    data['day_of_week'] = data.timestamp.dt.dayofweek.astype(int)
    
    data['bought_minus_built'] = data.timestamp.dt.year.astype(int) - data['build_year']
    
    data.loc[data['full_sq']==data['life_sq'],'life_sq'] = np.nan
    return data

def fact_binary(data):
    data = data.applymap(lambda x: x if x!='yes' else 1)
    data = data.applymap(lambda x: x if x!='no' else 0)
    data = data.applymap(lambda x: x if x!='OwnerOccupier' else 0)
    data = data.applymap(lambda x: x if x!='Investment' else 1)
    return data
class sep_estimator:
    def __init__(self,params, rounds):
        self.rounds = rounds
        self.params = params
        self.owner_lgb = None
        self.invest_lgb = None

    def preprocess_owner(self,data,mode):
        assert (data['product_type'].values==0).all()
        data = data.drop('timestamp',axis=1)
        return data
    
    def preprocess_investment(self,data,mode):
        if mode=='predict':
            assert (data['product_type'].values==1).all()
        data = data.drop('timestamp',axis=1)
        return data
    
    def fit(self,X,y):
        X1 = X[X['product_type']==0]
        X2 = X
        X1 = self.preprocess_owner(X1,'train')
        y1 = y.loc[X1.index.values]
        X2 = self.preprocess_investment(X2,'train')
        y2 = y
        
        if len(X1)>0:   
            other_train_lgb = lgb.Dataset(X1,y1)
            
            self.owner_lgb = lgb.train(self.params,other_train_lgb,num_boost_round=self.rounds)

        if len(X2)>0:
            inv_train_lgb = lgb.Dataset(X2,y2)
            self.invest_lgb = lgb.train(self.params,inv_train_lgb,num_boost_round=self.rounds)


    def predict(self,X):
        X1 = X[X['product_type']==0]
        X2 = X[X['product_type']==1]
        owner_index = X1.index.values
        investment_index = X.index.drop(owner_index).values
        X1 = self.preprocess_owner(X1,'predict')
        X2 = self.preprocess_investment(X2,'predict')
        res = pd.DataFrame(index=X.index)
        if len(X1)>0:
            pred1 = self.owner_lgb.predict(X1)
            res.loc[owner_index,0] = pred1
        if len(X2)>0:
            pred2 = self.invest_lgb.predict(X2)
            res.loc[investment_index,0] = pred2
        return res[0].values.flatten()

In [2]:
data = pd.read_csv('./csvs/data/train_clean.csv',index_col='id',parse_dates=['timestamp'])
data = fact_binary(data)
data = preprocess(data)
data = data.drop(['sub_area','ecology'],axis=1)
train = data

C:\Users\wongd\AppData\Local\Temp\ipykernel_27868\400300049.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='yes' else 1)
C:\Users\wongd\AppData\Local\Temp\ipykernel_27868\400300049.py:61: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='no' else 0)
C:\Users\wongd\AppData\Local\Temp\ipykernel_27868\400300049.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='OwnerOccupier' else 0)
C:\Users\wongd\AppData\Local\Temp\ipykernel_27868\400300049.py:63: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='Investment' else 1)


In [3]:
test = pd.read_csv('./csvs/data/test_clean.csv', parse_dates=['timestamp'],index_col='id')
test = preprocess(test)
test = fact_binary(test)
test = test.drop(['ecology','sub_area'],axis=1)
test.loc[test['product_type'].isnull(),'product_type'] = 1

test.loc[test['full_sq'].isnull(),'full_sq'] = 50


C:\Users\wongd\AppData\Local\Temp\ipykernel_27868\400300049.py:60: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='yes' else 1)
C:\Users\wongd\AppData\Local\Temp\ipykernel_27868\400300049.py:61: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='no' else 0)
C:\Users\wongd\AppData\Local\Temp\ipykernel_27868\400300049.py:62: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='OwnerOccupier' else 0)
C:\Users\wongd\AppData\Local\Temp\ipykernel_27868\400300049.py:63: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data = data.applymap(lambda x: x if x!='Investment' else 1)


In [5]:
#
#    Price level multipliers here. I've copied it from Andy Harless script:
#    https://www.kaggle.com/aharless/exercising-the-exorcism
#
#

rate_2015_q2 = 1
rate_2015_q1 = rate_2015_q2 / 0.9932
rate_2014_q4 = rate_2015_q1 / 1.0112
rate_2014_q3 = rate_2014_q4 / 1.0169
rate_2014_q2 = rate_2014_q3 / 1.0086
rate_2014_q1 = rate_2014_q2 / 1.0126
rate_2013_q4 = rate_2014_q1 / 0.9902
rate_2013_q3 = rate_2013_q4 / 1.0041
rate_2013_q2 = rate_2013_q3 / 1.0044
rate_2013_q1 = rate_2013_q2 / 1.0104  # This is 1.002 (relative to mult), close to 1:
rate_2012_q4 = rate_2013_q1 / 0.9832  #     maybe use 2013q1 as a base quarter and get rid of mult?
rate_2012_q3 = rate_2012_q4 / 1.0277
rate_2012_q2 = rate_2012_q3 / 1.0279
rate_2012_q1 = rate_2012_q2 / 1.0279
rate_2011_q4 = rate_2012_q1 / 1.076
rate_2011_q3 = rate_2011_q4 / 1.0236
rate_2011_q2 = rate_2011_q3 / 1
rate_2011_q1 = rate_2011_q2 / 1.011


# train 2015
train['average_q_price'] = 1

train_2015_q2_index = train.loc[train['timestamp'].dt.year == 2015].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2015_q2_index, 'average_q_price'] = rate_2015_q2

train_2015_q1_index = train.loc[train['timestamp'].dt.year == 2015].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2015_q1_index, 'average_q_price'] = rate_2015_q1


# train 2014
train_2014_q4_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2014_q4_index, 'average_q_price'] = rate_2014_q4

train_2014_q3_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2014_q3_index, 'average_q_price'] = rate_2014_q3

train_2014_q2_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2014_q2_index, 'average_q_price'] = rate_2014_q2

train_2014_q1_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2014_q1_index, 'average_q_price'] = rate_2014_q1


# train 2013
train_2013_q4_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2013_q4_index, 'average_q_price'] = rate_2013_q4

train_2013_q3_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2013_q3_index, 'average_q_price'] = rate_2013_q3

train_2013_q2_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2013_q2_index, 'average_q_price'] = rate_2013_q2

train_2013_q1_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2013_q1_index, 'average_q_price'] = rate_2013_q1


# train 2012
train_2012_q4_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2012_q4_index, 'average_q_price'] = rate_2012_q4

train_2012_q3_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2012_q3_index, 'average_q_price'] = rate_2012_q3

train_2012_q2_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2012_q2_index, 'average_q_price'] = rate_2012_q2

train_2012_q1_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2012_q1_index, 'average_q_price'] = rate_2012_q1


# train 2011
train_2011_q4_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2011_q4_index, 'average_q_price'] = rate_2011_q4

train_2011_q3_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2011_q3_index, 'average_q_price'] = rate_2011_q3

train_2011_q2_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2011_q2_index, 'average_q_price'] = rate_2011_q2

train_2011_q1_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2011_q1_index, 'average_q_price'] = rate_2011_q1

train['price_doc'] = train['price_doc'] * train['average_q_price']


#########################################################################################################

X = train.drop(["price_doc", "average_q_price"],axis=1)
y = train['price_doc']

C:\Users\wongd\AppData\Local\Temp\ipykernel_27868\1453253302.py:34: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.0068465565847766' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train.loc[train_2015_q1_index, 'average_q_price'] = rate_2015_q1


In [6]:
RS=1
ROUNDS = 1300
params = {
    'objective': 'regression', 
    'metric': 'rmse', 
    'boosting': 'gbdt', 
    'verbose': -1, 
    'learning_rate': 0.01, 
    'num_leaves': 32, 
    'bagging_fraction': 0.9, 
    'bagging_freq': 1, 
    'bagging_seed': 1, 
    'feature_fraction': 0.6, 
    'feature_fraction_seed': 1, 
    'max_bin': 64, 'max_depth': 7, 
    'num_rounds': 1500
}
est = sep_estimator(params=params,rounds=ROUNDS)
est.fit(X,y)


c:\Users\wongd\OneDrive\Documents\GitHub\machine_learning_project\cz4041project\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\wongd\OneDrive\Documents\GitHub\machine_learning_project\cz4041project\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [7]:
pred_test = np.zeros(shape=test.values.shape[0])
pr = est.predict(test)
pred_test += pr

subm = pd.read_csv('./csvs/data/sample_submission.csv')
subm['price_doc'] = pred_test
subm.to_csv('./csvs/submissions/model_3_output_lgb.csv',index=False)
subm.head()

,id,price_doc
0,30474,5.432429e+06
1,30475,8.316235e+06
2,30476,5.264321e+06
3,30477,6.338690e+06
4,30478,5.161522e+06
